pip install langchain langchain-google-genai google-generativeai python-dotenv

**Notes / secure alternatives**:
- It's better not to store API keys directly in source files in production.
- Alternatives: store key in an environment variable (set outside the repo), use a `.env` file with `python-dotenv`, or a secrets manager (AWS Secrets Manager, Azure Key Vault, GCP Secret Manager).
- Example using `python-dotenv`:

```python
from dotenv import load_dotenv
import os
load_dotenv()  # reads .env in cwd
key = os.getenv('GEMINI_KEY')
```

In [54]:
# --- Imports ---
from dotenv import load_dotenv
import os
# --- Load API key ---
load_dotenv()  # reads .env file
api_key_gemini = os.getenv("GEMINI_KEY")
api_key_gorq = os.getenv("GORQ_KEY")


In [55]:
from langchain_google_genai import GoogleGenerativeAI
# --- Initialize LLM ---
llm = GoogleGenerativeAI(                  
    model="gemini-2.0-flash",  # model selection
    google_api_key=api_key_gemini, #auth key(API key from .env file)
    temperature=0.6 # creativity level (0-1) 0 strait forward, 1 very creative(may get false information)
)

In [56]:
# --- Simple test ---
res = llm.invoke("Tell me a joke about engineers and bugs") #simple call usnig invoke method
print(res)

Why did the engineer refuse to swat the bug in his code?

Because he heard it was a feature!



pip install langchain-groq

In [90]:
from langchain_groq import ChatGroq

# Set your Groq API key as an environment variable or pass it directly
# os.environ["GROQ_API_KEY"] = "your_groq_api_key"

# Initialize the ChatGroq model
# llm = ChatGroq(
#     model="mixtral-8x7b-32768",  # Specify the Groq model you want to use
#     temperature=0.0,
#     # You can add other parameters like max_tokens, max_retries, etc.
# )
llm2 = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=api_key_gorq,
    temperature=0.6,
    max_tokens=50,
)

In [58]:
# --- Simple test ---
res = llm2.invoke("Tell me a joke about engineers and bugs") #simple call usnig invoke method
print (res)

content='Why did the engineer bring a ladder to the bug party?\n\nBecause he heard the drinks were on the house.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 43, 'total_tokens': 66, 'completion_time': 0.033236929, 'prompt_time': 0.002884208, 'queue_time': 0.044458272, 'total_time': 0.036121137}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--77a15816-5ed6-429f-8929-8a9653f078ab-0' usage_metadata={'input_tokens': 43, 'output_tokens': 23, 'total_tokens': 66}


In [59]:
print(res.content)

Why did the engineer bring a ladder to the bug party?

Because he heard the drinks were on the house.


**Langchain PrompTemplate**

In [60]:
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage

prompt = PromptTemplate(
    input_variables=["product"], #varibale name, here product
    template="What is a good name for a company that makes {product}? IN ONE WORD PLEASE." #prompt template here product will replace with input variable value
)
prompt.format(product="Coffee") # for example if we pass coffee it will return "What is a good name for a company that makes Coffee?"

'What is a good name for a company that makes Coffee? IN ONE WORD PLEASE.'

**LangChain - LLMChian**

In [61]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm2, prompt=prompt) # llm is the model and prompt is the prompt template
response = chain.run("Displays for verius device") # passing value to input variable product
print (response)

Here are a few suggestions for a one-word company name:

1. Displayo
2. Vizion
3. Viewmax
4. Pixio
5. Spectra

Let me know if you need more suggestions or if you have


Simple Sequntial chain

In [74]:
from langchain.prompts import PromptTemplate

prompt1 = PromptTemplate(
    input_variables=["cuisine"], #varibale name, here product
    template='I want to open new restorent, Suggest me only one the best fancy name which servs "{cuisine}" food.' 
)
print(prompt1.format(cuisine="Thai")) # for example if we pass coffee it will return "What is a good name for a company that makes Coffee?"
chain1 = LLMChain(llm=llm2, prompt=prompt1) # llm is the model and prompt is the prompt template


I want to open new restorent, Suggest me only one the best fancy name which servs "Thai" food.


In [86]:
prompt2 = PromptTemplate(
    input_variables=["resorent_name"], #varibale name, here resorent-name
    template="Give me only 'one short' tagline for the '{resorent_name}' restorent."
    #template='' #prompt template here product will replace with input variable value
)
print(prompt2.format(resorent_name="Spicy Delight")) # for example if we pass Spicy Delight it will return "Give me a tagline for the "Spicy Delight" restorent."
chain2 = LLMChain(llm=llm2, prompt=prompt2) # llm is the model and prompt is the prompt template

Give me only 'one short' tagline for the 'Spicy Delight' restorent.


In [87]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[chain1, chain2]) # here we can pass multiple chains in list, but we have only one chain
response = chain.run("Japanese")
print(response)

Here's a short tagline for "Sakura Kokoro": 

"Where beauty blooms, traditions unfold"


SequentialChain

In [ ]:
from langchain.chains import LLMChain, SequentialChain

# --- Chain 1: suggest a fancy restaurant name ---
prompt1 = PromptTemplate(
    input_variables=["cuisine"],
    template="Suggest one fancy, short restaurant name for {cuisine} food."
)
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key="restaurant_name")

# --- Chain 2: suggest menu items based on restaurant name ---
prompt2 = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggest 3 signature dishes for a restaurant named '{restaurant_name}'. Keep it concise."
)
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key="menu")

# --- Chain 3: suggest slogan based on restaurant name ---
prompt3 = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Write one short catchy slogan for '{restaurant_name}'. Keep it under 8 words."
)
chain3 = LLMChain(llm=llm, prompt=prompt3, output_key="slogan")

# --- Combine all in a proper SequentialChain ---
overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3], #order matters
    input_variables=["cuisine"],   # what we need to provide as input
    output_variables=["restaurant_name", "menu", "slogan"], # what we want to get in the end
    verbose=False  # turn this OFF, if true you will see the prompt and response for each chain step, useful for debugging
)

result = overall_chain.invoke({"cuisine": "Japanese"}) # dictionary input because there can be multiple input variables

print("🍣 Restaurant Name:", result["restaurant_name"].strip())
print("\n📜 Menu:\n", result["menu"].strip())
print("\n💡 Slogan:", result["slogan"].strip())

🍣 Restaurant Name: **Umami**

📜 Menu:
 1.  **Umami Bomb Ramen:** Rich pork broth, slow-braised pork belly, marinated egg, black garlic oil, and a secret umami-packed chili paste.
2.  **Seared Scallops with Shiitake Risotto:** Perfectly seared scallops atop creamy shiitake mushroom risotto, finished with a truffle-soy glaze.
3.  **Miso-Glazed Black Cod:** Delicate black cod marinated in a sweet and savory miso glaze, served with pickled ginger and steamed bok choy.

💡 Slogan: **Umami: The fifth taste, deliciously amplified.**
